# Importations des données

Les classes :

0. Chat
1. Lynx
2. Loup
3. Coyote
4. Jaguar
5. Guépard
6. Chimpanzé
7. Orang-Outan
8. Hamster
9. Cochon d'Inde
10. background

In [ ]:
from utils.dataloader import load_data6,load_data10,load_test_data
from utils.metrics import *
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import numpy as np
import os
import re
from utils.affichage import *

## Données de test

In [ ]:
x_test,y_test = load_test_data()
print(x_test.shape,y_test.shape)

In [ ]:
index = 4
plt.figure()
plt.axis("off")
plt.imshow(x_test[index,:,:,:].astype('uint8'))
plt.figure()
plt.axis("off")
plt.imshow(y_test[index,:,:,10])

## Données annotées par nous (6 classes, 311 données, environ 50 images par classe)

In [ ]:
x6,y6 = load_data6()
print(x6.shape,y6.shape)

In [ ]:
index = 96
plt.figure()
plt.axis("off")
plt.imshow(x6[index,:,:,:].astype('uint8'))
plt.figure()
plt.axis("off")
plt.imshow(y6[index,:,:,3])

## Ne garder que les données de test correspondant à nos classes

In [ ]:
y_test_filtered = np.empty((0,64,64,11))
indices_filtered = []
for i in range(len(x_test)):
    if np.max(y_test[i,:,:,[2,3,4,5,6,7]])>0:
        y_test_filtered = np.append(y_test_filtered,np.expand_dims(y_test[i],0),0)
        indices_filtered.append(i)
x_test_filtered = x_test[indices_filtered]
print(x_test_filtered.shape,y_test_filtered.shape)

## (FACULTATIF) Load toutes les données non annotées (seulement utile pour le teacher-student)

In [ ]:
# Chemin vers la base de données
path = "./animals/unlabelled/"
# Indice d'ajout de données dans les variables x et y 
i = 0
# Préparation des structures de données pour x et y
x_unlabeled = np.zeros((50000, 64, 64, 3))

# Parcours des fichiers (classés) du répertoire
dirs = os.listdir(path)

# Trier les fichiers par nom alphanumérique
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)
dirs = sorted_alphanumeric(dirs)

for item in dirs:
  # Image : on va remplir la variable x
  # Lecture de l'image
  img = Image.open(path + item)
  # Remplissage de la variable x
  x_unlabeled[i] = np.asarray(img)
  i = i+1

x_unlabeled = x_unlabeled.astype('uint8')

## (FACULTATIF) Load certaines données non annotées mais filtrées pour nos 6 classes (seulement utile pour le teacher-student)

In [ ]:
# Chemin vers la base de données
path = "./animals/unlabelled_filtered/"

# Préparation des structures de données pour x et y
x_unlabeled_filtered = np.empty((0, 64, 64, 3))

# Parcours des fichiers (classés) du répertoire
dirs = os.listdir(path)

# Trier les fichiers par nom alphanumérique
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)
dirs = sorted_alphanumeric(dirs)

for item in dirs:
  # Image : on va remplir la variable x
  # Lecture de l'image
  img = Image.open(path + item)
  # Remplissage de la variable x
  x_unlabeled_filtered = np.append(x_unlabeled_filtered,np.expand_dims(np.asarray(img),0),axis=0)

x_unlabeled_filtered = x_unlabeled_filtered.astype('uint8')

print(x_unlabeled_filtered.shape)

# Réseau et entraînement

## Création du réseau

In [ ]:
from tensorflow import keras
from keras.layers import UpSampling2D, concatenate
from keras import models
from keras.layers import Conv2D, MaxPooling2D, Dropout, Input
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l1_l2

l1 = 0
l2 = 0

reg = l1_l2(l1=l1,l2=l2)

def create_unet(image_size=64):
  input_layer=Input((image_size, image_size, 3))

  conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=reg)(input_layer)
  conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=reg)(conv1)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  
  conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=reg)(pool1)
  conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=reg)(conv2)
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
  
  conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=reg)(pool2)
  conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=reg)(conv3)
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
  
  conv4 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=reg)(pool3)
  conv4 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=reg)(conv4)
  drop4 = Dropout(0.5)(conv4)
  pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

  conv5 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=reg)(pool4)
  conv5 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=reg)(conv5)
  drop5 = Dropout(0.5)(conv5)

  up6 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
  merge6 = concatenate([drop4,up6], axis = 3)
  conv6 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=reg)(merge6)
  conv6 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

  up7 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
  merge7 = concatenate([conv3,up7], axis = 3)
  conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=reg)(merge7)
  conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=reg)(conv7)
  
  up8 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
  merge8 = concatenate([conv2,up8], axis = 3)
  conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=reg)(merge8)
  conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=reg)(conv8)

  up9 = Conv2D(32, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
  merge9 = concatenate([conv1,up9], axis = 3)
  conv9 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=reg)(merge9)
  conv9 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=reg)(conv9)
  conv10 = Conv2D(11, 1, activation = 'softmax')(conv9)


  model = Model(input_layer, conv10)

  return model

In [ ]:
model6 = create_unet(image_size=64)
model6.summary()

## Augmentation des données

In [ ]:
from albumentations import (Compose, RandomBrightness, RandomContrast, RandomGamma, ShiftScaleRotate, CenterCrop, HorizontalFlip, RandomSizedCrop, Rotate, RandomScale, Resize )

AUGMENTATIONS_TRAIN = Compose([
    ShiftScaleRotate(p=0.5),
    RandomContrast(limit=0.2, p=0.5),
    RandomGamma(gamma_limit=(80, 120), p=0.5),
    RandomBrightness(limit=0.2, p=0.5),
    HorizontalFlip(p=0.5)
])

AUGMENTATIONS_TRAIN2 = Compose([
    ShiftScaleRotate(p=1),
    RandomContrast(limit=0.2, p=1),
    RandomGamma(gamma_limit=(80, 120), p=1),
    RandomBrightness(limit=0.2, p=1),
    HorizontalFlip(p=0.5)
])

AUGMENTATIONS_TRAIN3 = Compose([
    Rotate(45,p=1),
    RandomScale(p=1),
    RandomContrast(limit=0.4, p=1),
    RandomGamma(gamma_limit=(80, 120), p=1),
    RandomBrightness(limit=0.3, p=1),
    HorizontalFlip(p=0.5),
    Resize(64,64,p=1)
])

AUGMENTATIONS_TRAIN4 = Compose([
    ShiftScaleRotate(p=0.9),
    RandomContrast(limit=0.2, p=0.9),
    RandomGamma(gamma_limit=(80, 120), p=0.9),
    RandomBrightness(limit=0.2, p=0.9),
    HorizontalFlip(p=0.5),
    RandomSizedCrop((48,48),64,64,p=0.9),
    Resize(64,64,p=1)
])

In [ ]:
from keras.utils.data_utils import Sequence

class LSPDSequence(Sequence):
    # Initialisation de la séquence avec différents paramètres
    def __init__(self, x_set, y_set, batch_size, augmentations,nb_augment):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.augment = augmentations
        self.indices1 = np.arange(x_set.shape[0]) 
        np.random.shuffle(self.indices1) # Les indices permettent d'accéder
        # aux données et sont randomisés à chaque epoch pour varier la composition
        # des batches au cours de l'entraînement
        self.nb_augment = nb_augment

    # Fonction calculant le nombre de pas de descente du gradient par epoch
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    # Application de l'augmentation de données à chaque image du batch et aux
    # cartes de probabilités associées
    def apply_augmentation(self, bx, by, n):
        """ n : number of transformations for each image in the batch"""
        batch_x = np.zeros((n*bx.shape[0],bx.shape[1],bx.shape[2],bx.shape[3]))
        batch_y = np.zeros((n*by.shape[0],by.shape[1],by.shape[2],by.shape[3]))
        # Pour chaque image du batch
        for i in range(len(bx)):
            masks = []
            # Les 14 masques associés à l'image sont rangés dans une liste pour 
            # pourvoir être traités par la librairie Albumentation
            for j in range(by.shape[3]):
                masks.append(by[i,:,:,j])

            img = bx[i]

            for t in range(n):
                # Application de l'augmentation à l'image et aux masques
                transformed = self.augment(image=img, masks=masks)
                batch_x[n*i+t] = transformed['image']
                batch_y_list = transformed['masks']

                # Reconstitution d'un tenseur à partir des masques augmentés
                for k in range(by.shape[3]):
                    batch_y[n*i+t,:,:,k] = batch_y_list[k]

        return batch_x/255, batch_y

    # Fonction appelée à chaque nouveau batch : sélection et augmentation des données
    def __getitem__(self, idx):
        batch_x = self.x[self.indices1[idx * self.batch_size:(idx + 1) * self.batch_size]]
        batch_y = self.y[self.indices1[idx * self.batch_size:(idx + 1) * self.batch_size]]
        
        batch_x, batch_y = self.apply_augmentation(batch_x, batch_y,self.nb_augment)

        return np.array(batch_x), np.array(batch_y)

    # Fonction appelée à la fin d'un epoch ; on randomise les indices d'accès aux données
    def on_epoch_end(self):
        np.random.shuffle(self.indices1)
        

In [ ]:
# On peut changer le type d'augmentation en changeant le paramètre augmentations
train_gen = LSPDSequence(x6.astype('uint8'), y6, 32, augmentations=AUGMENTATIONS_TRAIN2,nb_augment=2)

In [ ]:
# On peut exécuter cette cellule plusieurs fois pour voir les différentes augmentations

%matplotlib inline
batch_x, batch_y = train_gen.__getitem__(0)
plt.axis("off")
plt.imshow((255*batch_x[10]).astype('uint8'))
plt.figure()

plt.axis("off")
plt.imshow(batch_y[10,:,:,10])

## Entraînement du réseau (6 classes)

In [ ]:
model6 = create_unet(image_size=64)

opt = keras.optimizers.Adam(learning_rate=3e-4) 

model6.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics='accuracy')

# mcp_save = ModelCheckpoint('model_weights/val_loss_min2/{epoch:02d}-{val_loss:.2f}.hdf5', save_best_only=True,save_weights_only=True, monitor='val_loss', mode='min')
mcp_save = ModelCheckpoint('model_weights/6classes_augment2_bs32_aug2.hdf5', save_best_only=True,save_weights_only=True, monitor='val_loss', mode='min')
# mcp_save = ModelCheckpoint('model_weights/all_epochs2/{epoch:02d}.hdf5', save_best_only=False,save_weights_only=True, monitor='val_accuracy', mode='max',save_freq=20*50)

In [ ]:
history = model6.fit(train_gen,
          epochs=100,validation_data=(x_test_filtered/255,y_test_filtered),
          callbacks=[mcp_save])

## Analyse de l'entraînement

In [ ]:
%matplotlib inline
plot_training_analysis(history)

## (FACULTATIF) On charge les poids d'un bon entraînement

In [ ]:
model6.load_weights('model_weights/6classes_augment2_bs32_aug2_valloss0.82.hdf5')
model6.evaluate(x6/255,y6),model6.evaluate(x_test_filtered/255,y_test_filtered)

## Matrice de confusion

In [ ]:
%matplotlib qt
model6.load_weights('model_weights/6classes_augment2_bs32_aug2_valloss0.82.hdf5')
confusion_matrix(model6,x_test_filtered,y_test_filtered)

## Afficher une prédiction avec l'image et la vérité terrain

In [ ]:
%matplotlib inline
# 20, 40
model6.load_weights('model_weights/6classes_augment2_bs32_aug2_valloss0.82.hdf5')
ind_img = 40
ind = 6

prediction = model6.predict(np.expand_dims(x_test_filtered[ind_img], axis=0))
plt.axis("off")
plt.imshow(prediction[0,:,:,ind])
plt.show()
plt.axis("off")
plt.imshow(x_test_filtered[ind_img].astype('uint8'))
plt.show()
plt.axis("off")
plt.imshow(y_test_filtered[ind_img,:,:,ind])
plt.show()

# Teacher-Student

## Entraîner les réseaux

In [ ]:
nb_students = 1 # nombre de students à entraîner (à chaque itération, le student devient le nouveau teacher)
nb_new_images = 100

teacher = models.load_model("model_weights/6classes_augment2_bs32_aug2_valloss0.82.hdf5")

for i in range(nb_students):

    indices_all = np.arange(len(x_unlabeled_filtered))
    np.random.shuffle(indices_all)
    indices_new_images = indices_all[:nb_new_images]

    x_new = x_unlabeled_filtered[indices_new_images]
    y_new = teacher.predict(x_new)

    x_mixed = np.append(x6,x_new,axis=0)
    y_mixed = np.append(y6,y_new,axis=0)

    train_gen = LSPDSequence(x_mixed.astype('uint8'), y_mixed, 32, augmentations=AUGMENTATIONS_TRAIN2,nb_augment=2)

    student = create_unet(image_size=64)
    opt = keras.optimizers.Adam(learning_rate=3e-4)

    student.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics='accuracy')

    mcp_save = ModelCheckpoint('model_weights/.student_'+str(nb_new_images)+'_'+str(i)+'.hdf5', save_best_only=True, monitor='val_loss', mode='min')
    
    history = student.fit(train_gen,
            epochs=100,validation_data=(x_test_filtered/255,y_test_filtered),
            callbacks=[mcp_save])
    

## Evaluer les performances

In [ ]:
model_evaluate = models.load_model("model_weights/.student1352_0.hdf5")
model_evaluate.evaluate(x_test_filtered/255,y_test_filtered)

## Matrice de confusion

In [ ]:
%matplotlib qt
confusion_matrix(model_evaluate,x_test_filtered,y_test_filtered)

## Regarder des prédictions sur l'ensemble de test

In [ ]:
%matplotlib inline

ind_img = 26
ind = 4

prediction = model_evaluate.predict(np.expand_dims(x_test_filtered[ind_img], axis=0))
plt.axis("off")
plt.imshow(prediction[0,:,:,ind])
plt.show()
plt.axis("off")
plt.imshow(x_test_filtered[ind_img].astype('uint8'))
plt.show()
plt.axis("off")
plt.imshow(y_test_filtered[ind_img,:,:,ind])
plt.show()

## Afficher des prédictions sur les données non labellisées

In [ ]:
ind_img = 18848
ind = 6

prediction = model_evaluate.predict(np.expand_dims(x_unlabeled[ind_img], axis=0))
plt.axis("off")
plt.imshow(prediction[0,:,:,ind])
plt.show()
plt.axis("off")
plt.imshow(x_unlabeled[ind_img].astype('uint8'))
plt.show()